# SLU14 - k-Nearest Neighbors (kNN)

In this notebook we will have exercises covering the following topics:

- k-Nearest Neighbours Algorithm
- A Primer on Distance
- Some considerations about kNN
- Using kNN

In [1]:
# Place any important imports at the top of the notebook when possible
import hashlib
import json
import math
import numpy as np
import os
import pandas as pd

from sklearn import datasets

## Distances

### Exercise 1

Define a function called `euclidean_distance`. This function should receive two arguments, `a` and `b`, which are numpy arrays with shape `(N,)`, where `N` is the number of dimensions of the inputs `a` and `b`.

If the two arrays don't have the same shape, return None.

In case the arguments are valid, return the euclidean distance between them.

Of course you know about the function [numpy.linalg.norm](https://docs.scipy.org/doc/numpy/reference/generated/numpy.linalg.norm.html), which does exactly what we're asking here, but please take this opportunity to really understand the euclidean distance! Feel free to use it to double check your answer.

In [33]:
# implement a function called euclidean_distance
from numpy import linalg as LA
def euclidean_distance(a, b):
    """
    Euclidean distance between two vectors.
    
    Parameters
    ----------
    a: numpy array with shape (N,)
    b: numpy array with shape (N,)
    
    Returns
    ----------
    distance: float
    """

    # YOUR CODE HERE
    if a.shape != b.shape:
        return None
    else:
        return math.sqrt(np.square(a-b).sum())

In [34]:
# Test case 1
a = np.array([1, 2, 4])
b = np.array([-1, 0, 4])

assert math.isclose(euclidean_distance(a, b), 2.8284, rel_tol=1e-03)


# Test case 2
a = np.array([1, 2])
b = np.array([-1, 0, 4])

assert euclidean_distance(a, b) is None
             

# Test case 3
a = np.array([1])
b = np.array([-1])

assert math.isclose(euclidean_distance(a, b), 2.0, rel_tol=1e-03)


# Test case 4
a = np.array([0, 0])
b = np.array([2, 3])

assert math.isclose(euclidean_distance(a, b), 3.6055, rel_tol=1e-03)


# Test case 5
a = np.array([0, 1, 2, 3, 4])
b = np.array([0, -1, -2, -3, -4])

assert math.isclose(euclidean_distance(a, b), 10.9544, rel_tol=1e-03)

### Exercise 2

Define a function called `dot_product`. This function should receive two arguments, `a` and `b`, which are numpy arrays with shape `(N,)`, where `N` is the number of dimensions of the inputs `a` and `b`.

You can assume the two arrays have the same shape.

The function should return the dot product between the arrays.

Of course you know about the function [numpy.dot](https://docs.scipy.org/doc/numpy/reference/generated/numpy.dot.html), , which does exactly what we're asking here, but please take this opportunity to really understand the dot product! Feel free to use it to double check your answer.

In [35]:
# implement a function called dot_product

def dot_product(a, b):
    """
    Dot product between two vectors.
    
    Parameters
    ----------
    a: numpy array with shape (N,)
    b: numpy array with shape (N,)
    
    Returns
    ----------
    dot_product: float
    """
    

    # YOUR CODE HERE
    return (a*b).sum()

In [36]:
tests = [
    {
        'input': [np.array([1, 2, 4]), np.array([-1, 0, 4])],
        'output_hash': 'e629fa6598d732768f7c726b4b621285f9c3b85303900aa912017db7617d8bdb'
    },
    {
        'input': [np.array([0, 0]), np.array([2, 3])],
        'output_hash': '5feceb66ffc86f38d952786c6d696c79c2dbc239dd4e91b46729d73a27fb57e9'
    },
    {
        'input': [np.array([0, 1, 2, 3, 4]), np.array([0, -1, -2, -3, -4])],
        'output_hash': '4cbaf3fbc9b6ccc6d363e9cac9d51c6d3012fc8991a30cbe952c5e92c7927d92'
    }
]

for test in tests:
    answer = dot_product(*test['input'])
    answer_hash = hashlib.sha256(bytes(str(answer), encoding='utf8')).hexdigest()
    
    assert answer_hash == test['output_hash']

### Exercise 3

Define a function called `cosine_distance`. This function should receive two arguments, `a` and `b`, which are numpy arrays with shape `(N,)`, where `N` is the number of dimensions of the inputs `a` and `b`.

You can assume the two arrays have the same shape.

The function should return the cosine distance between the arrays.

Of course you know about the function [scipy.distance.cosine](https://docs.scipy.org/doc/scipy-0.14.0/reference/generated/scipy.spatial.distance.cosine.html), which does exactly what we're asking here, but please take this opportunity to really understand the cosine distance! Feel free to use it to double check your answer.

After you've implemented the function, take a moment to think what values can the cosine distance function return.

In [41]:
# implement a function called cosine_distance

def cosine_distance(a, b):
    """
    Cosine distance between two vectors.
    
    Parameters
    ----------
    a: numpy array with shape (N,)
    b: numpy array with shape (N,)
    
    Returns
    ----------
    cosine_distance: float
    """

    norm_a = euclidean_distance(np.zeros(len(a)).ravel(),a)
    norm_b = euclidean_distance(np.zeros(len(b)).ravel(),b)
    
    # YOUR CODE HERE
    return 1 - dot_product(a,b)/(norm_a*norm_b)

In [42]:
# Test case 1
a = np.array([1, 2, 4])
b = np.array([-1, 0, 4])

assert math.isclose(cosine_distance(a, b), 0.2061, rel_tol=1e-03)


# Test case 2
a = np.array([0, 1])
b = np.array([1, 0])

assert math.isclose(cosine_distance(a, b), 1.0, rel_tol=1e-03)


# Test case 3
a = np.array([0, 1, 2, 3, 4])
b = np.array([0, -1, -2, -3, -4])

assert math.isclose(cosine_distance(a, b), 2.0, rel_tol=1e-03)

## Implementing the kNN algorithm

By hand! Let's do this!

![lets_do_this](media/lets_do_this.gif)

### Exercise 4

The first step is to implement a function that computes a distance between one point and each other point in a dataset.

Let's implement a function called `compute_distances`, that:

* receives three arguments:
    * x, which is a numpy array with shape (d,)
    * dataset, which is a numpy array with shape (N, d), where N is the dataset size
    * distance_type, which can be 'euclidean', 'cosine', 'dot'
* computes the distance between x and all the points in the dataset. You should choose the right distance function, depending on the distance_function value. You can either use the functions that we've implemented above, or import them from numpy/scipy
* returns a numpy array of shape (N,) with the computed distances

In [49]:
# implement a function called compute_distances

def compute_distances(x, dataset, distance_type):
    """
    Computes a distance between a point and all the other points in a dataset.
    Supported distance functions are: euclidean, dot, cosine.
    
    Parameters
    ----------
    x: numpy array with shape (d,)
    dataset: numpy array with shape (N, d)
    distance_type: string
    
    Returns
    ----------
    distances: numpy array with shape (N,)
    """
    print(np.fromiter((euclidean_distance(x,xi) for xi in dataset), x.dtype))
    # YOUR CODE HERE
    return np.fromiter((euclidean_distance(x,xi) for xi in dataset), x.dtype)

In [50]:
dataset = datasets.load_iris().data
x = np.array([4.9, 3.0, 6.1, 2.2])

# Testing with euclidean distance
distances = compute_distances(x, dataset, 'euclidean')

assert isinstance(distances, np.ndarray), "The function should return a numpy array!"
assert distances.shape == (150,), "The returned numpy array has the wrong shape!"
assert math.isclose(distances[13], 5.456189, rel_tol=1e-03), "The returned numpy array has the wrong values!"
assert math.isclose(distances[47], 5.120546, rel_tol=1e-03), "The returned numpy array has the wrong values!"
assert math.isclose(distances[112], 1.994993, rel_tol=1e-03), "The returned numpy array has the wrong values!"

# Testing with dot product distance
distances = compute_distances(x, dataset, 'dot')

assert isinstance(distances, np.ndarray), "The function should return a numpy array!"
assert distances.shape == (150,), "The returned numpy array has the wrong shape!"
assert math.isclose(distances[13], 37.0, rel_tol=1e-03), "The returned numpy array has the wrong values!"
assert math.isclose(distances[47], 41.12, rel_tol=1e-03), "The returned numpy array has the wrong values!"
assert math.isclose(distances[112], 80.49, rel_tol=1e-03), "The returned numpy array has the wrong values!"

# Testing with cosine distance
distances = compute_distances(x, dataset, 'cosine')

assert isinstance(distances, np.ndarray), "The function should return a numpy array!"
assert distances.shape == (150,), "The returned numpy array has the wrong shape!"
assert math.isclose(distances[13], 0.202958, rel_tol=1e-03), "The returned numpy array has the wrong values!"
assert math.isclose(distances[47], 0.17874, rel_tol=1e-03), "The returned numpy array has the wrong values!"
assert math.isclose(distances[112], 0.02015, rel_tol=1e-03), "The returned numpy array has the wrong values!"

[5.13614642 5.10783712 5.20768663 5.02593275 5.14392846 4.8641546
 5.09411425 5.03289181 5.13322511 5.05766745 5.08920426 4.94165964
 5.14878626 5.45618915 5.46076918 5.19615242 5.22876658 5.09803884
 4.9244289  5.0447993  4.8754487  4.99299509 5.51905789 4.73075047
 4.67011777 4.92544414 4.8641546  5.04975247 5.13225097 4.93254498
 4.92645917 4.98096376 5.18362807 5.28109837 5.0169712  5.29716906
 5.25832673 5.18266341 5.22398315 5.03587132 5.18748494 5.22494019
 5.22781025 4.80312398 4.64327471 5.07050293 4.99299509 5.12054685
 5.08035432 5.11761663 2.65518361 2.310844   2.43721152 2.463737
 2.310844   2.01246118 2.0904545  3.10483494 2.44131112 2.37907545
 3.03479818 2.25831796 2.77488739 2.01246118 2.74954542 2.60384331
 1.88148877 2.51793566 2.3194827  2.60576284 1.7        2.58843582
 2.03469899 2.10713075 2.51197134 2.53377189 2.44540385 2.16794834
 2.06639783 3.         2.68700577 2.81424946 2.596151   1.63095064
 1.81659021 2.07123152 2.38746728 2.47991935 2.30217289 2.4145392

AssertionError: The returned numpy array has the wrong values!

### Exercise 5

Now that we have a function that computes the distance between one point and all the other points in a dataset, we need to select the point's nearest neighbours, which are the points in the dataset for which the distance is the minimum.

In this exercise, you'll implement a function called `select_nearest_neighbours`, that:

* receives two arguments:
    * distances, which is a numpy array with distances (like the one returned in the previous question)
    * k, which is the number of nearest neighbours that we want to consider
* gets the indexes of the k smallest distances
* returns a numpy array of shape (k,) with those indexes

Hint: check [numpy.argsort](https://docs.scipy.org/doc/numpy/reference/generated/numpy.argsort.html).

In [ ]:
# implement a function called select_nearest_neighbours

def select_nearest_neighbours(distances, k):
    """
    Selects the k nearest neighbours
    
    Parameters
    ----------
    distances: numpy array with shape (N,)
    k: int, the number of nearest neighbours we want to consider
    
    Returns
    ----------
    indexes: numpy array with shape (k,)
    """

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
# This is to make the random predictable
np.random.seed(42)

# Test case 1
knn = select_nearest_neighbours(np.random.rand(150), 3)
assert knn.shape == (3,)
assert hashlib.sha256(bytes(knn[2])).hexdigest() == '01d448afd928065458cf670b60f5a594d735af0172c8d67f22a81680132681ca'

# Test case 2
knn = select_nearest_neighbours(np.random.rand(49), 10)
assert knn.shape == (10,)
assert hashlib.sha256(bytes(knn[5])).hexdigest() == '11e431c215c5bd334cecbd43148274edf3ffdbd6cd6479fe279577fbe5f52ce6'

### Exercise 6

Now that we have a function that gets the indexes of the k nearest neighbours, we need to get the values of those neighbours, so that afterwards we can predict the label for our point.

In this exercise, you'll implement a function called `get_nn_labels`, that:

* receives two arguments:
    * neighbour_indexes, which are the indexes of the k nearest neighbours (like the output of the last function)
    * y_train, which is a numpy array with the targets from the a training set
* gets the values from y_train using the indexes from neighbour_indexes
* returns a numpy array of shape (k,) with those values

In [ ]:
# implement a function called get_nn_labels

def get_nn_labels(y_train, neighbour_indexes):
    """
    Selects the label values from the k nearest neighbours
    
    Parameters
    ----------
    y_train: numpy array with shape (N,)
    neighbour_indexes: numpy array with shape (k,)
    
    Returns
    ----------
    labels: numpy array with shape (k,)
    """

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
np.random.seed(42) 

# Test case 1
answer = get_nn_labels(np.random.rand(150), np.random.randint(0, 3, 3))
assert answer.shape == (3,)
assert math.isclose(answer[0], 0.37454, rel_tol=1e-03)

# Test case 2
answer = get_nn_labels(np.random.rand(10), np.random.randint(0, 3, 7))
assert answer.shape == (7,)
assert math.isclose(answer[3], 0.44778, rel_tol=1e-03)

### Exercise 7

Next we need to predict a label for our point based on the labels of the nearest neighbours.

In this exercise, you'll implement a function called `predict_label_majority`, that:

* receives one argument:
    * nn_labels, which are the labels from the k nearest neighbours
* returns the most frequent label

In [ ]:
# implement a function called predict_label_majority

def predict_label_majority(nn_labels):
    """
    Selects the most frequent label in nn_labels
    
    Parameters
    ----------
    nn_labels: numpy array with shape (k,)
    
    Returns
    ----------
    label: int
    """

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
np.random.seed(42) 

# Test case 1
answer = predict_label_majority(np.random.randint(0, 3, 3))
assert isinstance(answer, int)
assert hashlib.sha256(bytes(answer)).hexdigest() == '96a296d224f285c67bee93c30f8a309157f0daa35dc5b87e410b78630a09cfc7'


# Test case 2
answer = predict_label_majority(np.random.randint(0, 3, 5))
assert isinstance(answer, int)
assert hashlib.sha256(bytes(answer)).hexdigest() == 'e3b0c44298fc1c149afbf4c8996fb92427ae41e4649b934ca495991b7852b855'

### Exercise 8

Finally we can put everything together and implement the knn classifier!

In this exercise, you'll implement a function called `knn_classifier`, that:

* receives five arguments:
    * x, which is a numpy array with shape (d,)
    * dataset, which is a numpy array with shape (N, d), where N is the dataset size
    * targets, which is a numpy array with shape (N,), that has the targets for each of the points in the dataset
    * k, which is the number of nearest neighbours our knn algorithm will consider
    * distance_function, which can be 'euclidean', 'cosine', 'dot'
* uses all the functions that we implemented above in order to implement a knn_classifier!

In [ ]:
# implement a function called knn_classifier

def knn_classifier(x, dataset, targets, k, distance_function):
    """
    Predicts the label for a single point, given a dataset, a value for k and a distance function
    
    Parameters
    ----------
    x: numpy array with shape (d,)
    dataset: numpy array with shape (N, d)
    targets: numpy array with shape (N,)
    k: int
    distance_function: string
    
    Returns
    ----------
    label: int
    """

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
dataset = datasets.load_iris().data
targets = datasets.load_iris().target
x = np.array([4.9, 3.0, 6.1, 2.2])

tests = [
    {
        'input': [x, dataset, targets, 3, 'euclidean'],
        'expected_value': 2
    },
    {
        'input': [x, dataset, targets, 5, 'dot'],
        'expected_value': 0
    },
    {
        'input': [x, dataset, targets, 1, 'cosine'],
        'expected_value': 2
    }
]

for test in tests:
    pred_label = knn_classifier(*test['input'])
    assert isinstance(pred_label, int), "The function should return an integer!"
    assert pred_label == test['expected_value'], "The returned int has the wrong value!"

Now that we've implemented a knn classifier, let's go a bit further and implement a knn regressor!

Luckily, we can reuse most of the functions we've already implemented!

Keep up the good work, we're almost there!

![almost_there](media/almost_there.gif)

### Exercise 9

As we explained in the learning notebook, the main difference between a knn classifier and a knn regressor is the way we choo
se the predicted label from the labels of the nearest neighbours.

For the classifier case we used a majority vote. In the regressor case, we want to use an the average value of the neighbours' labels.

In this exercise, you'll implement a function called `predict_label_average`, that:

* receives one argument:
    * nn_labels, which are the labels from the k nearest neighbours
* returns the average of the nearest neighbours' labels

In [ ]:
# implement a function called predict_label_majority

def predict_label_average(nn_labels):
    """
    Gets the average of the labels from the nearest neighbours
    
    Parameters
    ----------
    nn_labels: numpy array with shape (k,)
    
    Returns
    ----------
    label: float
    """

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
np.random.seed(42) 

label_average = predict_label_average(np.random.rand(3))
assert isinstance(label_average, float)
assert math.isclose(label_average, 0.685749, rel_tol=1e-04)

label_average = predict_label_average(np.random.rand(5))
assert isinstance(label_average, float)
assert math.isclose(label_average, 0.3669862, rel_tol=1e-04)

### Exercise 10

And we're ready to implement the knn regressor!

In this exercise, you'll implement a function called `knn_regressor`, that:

* receives five arguments:
    * x, which is a numpy array with shape (d,)
    * dataset, which is a numpy array with shape (N, d), where N is the dataset size, and d is the number of dimensions that the points in the dataset have
    * targets, which is a numpy array with shape (N,), that has the targets for each of the points in the dataset
    * k, which is the number of nearest neighbours our knn algorithm will consider
    * distance_function, which can be 'euclidean', 'cosine', 'dot'
* uses all the functions that we implemented above in order to implement a knn_regressor!

In [ ]:
# implement a function called knn_classifier

def knn_regressor(x, dataset, targets, k, distance_function):
    """
    Predicts the label for a single point, given a dataset, a value for k and a distance function
    
    Parameters
    ----------
    x: numpy array with shape (d,)
    dataset: numpy array with shape (N, d)
    targets: numpy array with shape (N,)
    k: int
    distance_function: string
    
    Returns
    ----------
    label: float
    """

    # YOUR CODE HERE
    raise NotImplementedError()

In [ ]:
np.random.seed(42)
dataset = datasets.load_diabetes().data
targets = datasets.load_diabetes().target
x = np.random.rand(10)

prediction = knn_regressor(x, dataset, targets, 3, 'euclidean')
assert isinstance(prediction, float)
assert math.isclose(prediction, 265.666, rel_tol=1e-04)

prediction = knn_regressor(x, dataset, targets, 5, 'dot')
assert isinstance(prediction, float)
assert math.isclose(prediction, 92.8, rel_tol=1e-04)

prediction = knn_regressor(x, dataset, targets, 1, 'cosine')
assert isinstance(prediction, float)
assert math.isclose(prediction, 264.0, rel_tol=1e-04)

**Well done!!!**

![we_did_it](media/we_did_it.gif)

Finally let's wrap this up with a couple of exercises on how to use scikit's knn models.

## Using scikit's knn models

### Exercise 11

Use a `KNeighborsClassifier` to create predictions for the [wine dataset](https://scikit-learn.org/stable/modules/generated/sklearn.datasets.load_wine.html#sklearn.datasets.load_wine).

Follow the instructions in the comments in the exercise cell.

In [ ]:
import numpy as np
import pandas as pd
import hashlib
import json

from scipy.spatial.distance import cosine
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsClassifier, KNeighborsRegressor
from sklearn.metrics import accuracy_score

In [ ]:
# We start by importing the dataset
data = datasets.load_wine()

# Now do a train test split, using the train_test_split function from scikit
# Use a test_size of 0.33 and a random_state of 42
# X_train, X_test, y_train, y_test = ...

# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
tests = [
    {
        'dataset_type': 'X_train',
        'dataset': X_train,
        'shape_hash': '4c89825fdaebd81d00f7b45dafa18ef7b1ba1ae7842ef7d783557c3c9ddf7a03'
    },
    {
        'dataset_type': 'X_test',
        'dataset': X_test,
        'shape_hash': 'f6c8d3daeed9b2c2df64a756aa8d44614e45f108d571247865e59315fbfe578f'
    },
    {
        'dataset_type': 'y_train',
        'dataset': y_train,
        'shape_hash': 'ea38832e303c9a40cfe8b6160fd949f7317febe9633fc7c8a1153aa5e7c2512e'
    },
    {
        'dataset_type': 'y_test',
        'dataset': y_test,
        'shape_hash': '8f332db1356cb4786b468baab31bd75565bd751de9d984c92f9defbcc3ef172d'
    },
]

for test in tests:
    shape_hash = hashlib.sha256(json.dumps(test['dataset'].shape).encode()).hexdigest()

    assert isinstance(test['dataset'], np.ndarray), f"{test['dataset_type']} should be a numpy array!"
    assert shape_hash == test['shape_hash'], "The returned numpy array has the wrong shape!"

In [ ]:
# Now instantiate a kNN Classifier with k=3, that uses the euclidean distance as distance function
# In scikit, the euclidean distance is the default one and goes by the name of 'minkowski'
# which is in fact a generalisation of the euclidean distance
# clf = ...
# YOUR CODE HERE
raise NotImplementedError()


# Get predictions for the test dataset
# y_pred = ...
# YOUR CODE HERE
raise NotImplementedError()

# Measure the accuracy of your solution using scikit's accuracy_score function
# accuracy = ...
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert isinstance(clf, KNeighborsClassifier)
assert clf.n_neighbors == 3
assert clf.metric == 'minkowski'

assert isinstance(y_pred, np.ndarray)
assert y_pred.shape == (59,)

assert isinstance(accuracy, float)
assert math.isclose(accuracy, 0.694915, rel_tol=1e-04)

## Exercise 12

Now we want to see the difference if we use the cosine distance instead of the euclidean distance.

Go through the same steps as the previous exercise, but use the cosine distance as the distance metric in the knn classifier.

In [ ]:
# Instantiate a kNN Classifier with k=3, that uses the cosine distance as distance function
# clf = ...
# YOUR CODE HERE
raise NotImplementedError()


# Get predictions for the test dataset
# y_pred = ...
# YOUR CODE HERE
raise NotImplementedError()

# Measure the accuracy of your solution using scikit's accuracy_score function
# accuracy = ...
# YOUR CODE HERE
raise NotImplementedError()

In [ ]:
assert isinstance(clf, KNeighborsClassifier)
assert clf.n_neighbors == 3
assert clf.metric == cosine

assert isinstance(y_pred, np.ndarray)
assert y_pred.shape == (59,)

assert isinstance(accuracy, float)
assert math.isclose(accuracy, 0.7796610, rel_tol=1e-04)

And we're done! Nice job ;)

![were_done](media/were_done.gif)